# Enron - How many POIs we can predict?

### Index

1. Set variables
2. Data cleansing
3. New features
4. Preparing the data for the split
5. Spliting data into train and test
6. Selecting features
7. Testing models
8. Tuning Algorithm Parameters

In [5]:
import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data, test_classifier
from sklearn import model_selection
from time import time
import pandas as pd
import pylab as pl
import numpy as np

from sklearn.cross_validation import StratifiedShuffleSplit
    
# Preprocessing
from sklearn.preprocessing import  MaxAbsScaler, StandardScaler, MinMaxScaler

#Models
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest,chi2, f_classif
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.tree import DecisionTreeClassifier

#Feature selection
from sklearn.decomposition import PCA,RandomizedPCA, TruncatedSVD
from sklearn.feature_selection import SelectKBest,chi2, f_regression


#pipeline
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

In the next steps the data will be cleaned, if you want to understand the process, check: 
https://github.com/liebycardoso/ML_Enron/blob/master/Enron_Data_Analysis.ipynb

### 1. Set variables

In [3]:
target_label = 'poi'                

financial_features_list = [
    'bonus',
    'deferral_payments',
    'deferred_income',
    'director_fees',
    'exercised_stock_options',
    'expenses',
    'loan_advances',
    'long_term_incentive',
    'other',
    'restricted_stock',
    'restricted_stock_deferred',
    'salary'
]

features_list = [target_label] + financial_features_list  

### 2. Data cleansing

In [4]:
with open("P:/Nanodegree/ML/ud120-projects/tools/final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

### Task 2: Remove outliers
outliers = ["TOTAL", "THE TRAVEL AGENCY IN THE PARK", "LOCKHART EUGENE E", "CHAN RONNIE"]
for outlier in outliers :
    data_dict.pop(outlier, 0)


def update_dict_value(key, items, values, dict_obj):
    index = 0
    for item in items:     
        dict_obj[key][item] = values[index]
        index += 1
    return dict_obj       

    
data_dict = update_dict_value(
              'BELFER ROBERT',
              ['deferred_income','deferral_payments', 'expenses', 
               'director_fees', 'total_payments', 'exercised_stock_options',
               'restricted_stock','restricted_stock_deferred',
               'total_stock_value'], 
              [-102500,'NaN',3285,102500, 3285,'NaN', 44093,-44093,'NaN'],
              data_dict)


data_dict = update_dict_value(
              'BHATNAGAR SANJAY',
              ['other', 'expenses', 'director_fees', 'total_payments',
               'exercised_stock_options','restricted_stock',
               'restricted_stock_deferred','total_stock_value'],
              ['NaN',137864, 'NaN', 137864, 15456290, 
               2604490, -2604490, 15456290],
               data_dict)


### 3. New features

In [6]:
for key in data_dict:
    key_values = data_dict[key]

    total_msg = (data_dict[key]['to_messages'] + 
                 data_dict[key]['from_messages'])
    
    total_poi_msg = (data_dict[key]['from_poi_to_this_person'] +
                     data_dict[key]['from_this_person_to_poi'] + 
                     data_dict[key]['shared_receipt_with_poi'])     
        
    try:
        data_dict[key]['message_poi_ratio'] = (float(total_poi_msg) / 
                                           float(total_msg))
    except:
        data_dict[key]['message_poi_ratio'] = "NaN"
        
    try:
        data_dict[key]['message_others_ratio'] = ((float(total_msg) - float(total_poi_msg)) / 
                                          float(total_msg))
    except:
        data_dict[key]['message_others_ratio'] = "NaN"

features_list = features_list + ['message_poi_ratio','message_others_ratio'] 


### 4. Preparing the data for the split

In [7]:
# Store to my_dataset for easy export below.
my_dataset = data_dict

# Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, remove_NaN=True, sort_keys = True)
labels, features = targetFeatureSplit(data)

### 5. Spliting data into train and test

In [8]:

# transformed version of X
# scaler = MinMaxScaler(feature_range=(0, 1))
# X_scaled = scaler.fit_transform(features)

# Split the dataset into train and test
features_train, features_test, labels_train, labels_test = model_selection.train_test_split(features, 
                                                                                            labels,  
                                                                                            test_size=0.3, 
                                                                                            random_state=42)

Since tje features have quite differente value ranges and some of them are discrete and some of them take continous values, I need to scale them first. Removing mean and dividing the standard deviation o features respectively, this one of the most commonly used preprocessing step.

# 6. Selecting features

For now I don't know exactly the number of features I want to use, so I decided to use PCA for identify the most important features and explain the variance.

In [9]:
for i in range(1,15):
    pca = PCA(svd_solver='auto', n_components=i)
    x = pca.fit(features_train).explained_variance_ratio_.sum()
    print i,":", x    

1 : 0.861553776249
2 : 0.950132015173
3 : 0.972382607444
4 : 0.984291217765
5 : 0.992901638092
6 : 0.997163221659
7 : 0.998629887029
8 : 0.999743067191
9 : 0.99987985018
10 : 0.999974804369
11 : 0.999991386498
12 : 1.0
13 : 1.0
14 : 1.0


In this decomposition, the vector array provided by ratio indicates that most of the information is concentrated into the first 2 compontents . You saw this same sort of result after the factor analysis. it's therefore possible to reduce the entire dataset to just two componentes, providing a reduction of noise and redundant information from the original dataset.

Let's see which variables are the most important:

In [11]:
pca = PCA(n_components=14)
pca.fit_transform(features_train)
pca_df = pd.DataFrame(zip(features_list[1:],
                          np.round(pca.explained_variance_ratio_, decimals=5)*100), 
                      columns=['feature','variance_ratio'])
pca_df.sort_values(by='variance_ratio',ascending = False)

,feature,variance_ratio
0,bonus,86.155
1,deferral_payments,8.858
2,deferred_income,2.225
3,director_fees,1.191
4,exercised_stock_options,0.861
5,expenses,0.426
6,loan_advances,0.147
7,long_term_incentive,0.111
8,other,0.014
9,restricted_stock,0.009


I really don't wanna that the features negatively impact my model performance. I'll spend more time on this and try SelecKbest for in this task. 

First of all, I scaled the data with MinMaxScaler (range 0, 1), then I applyied SelectKBest chi squared statistical test, cause this test need non-negative features. After that I used SelectKBest ANOVA F-value statistical test on the raw data, in a way I could compare the results.

In [12]:
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)

selector = SelectKBest(chi2, k='all').fit(features_scaled, labels)
k_best = SelectKBest(f_classif,k='all').fit(features, labels)    

# Format values
kbest_pd = pd.DataFrame(zip(features_list[1:],
                            k_best.scores_, # K best score                            
                            selector.scores_), # chi2
                        columns = ['feature','anova', 'chi2'])
print "SelectKBest: "
print kbest_pd.sort_values(by='anova',ascending = False)

SelectKBest: 
                      feature      anova      chi2
4     exercised_stock_options  22.087532  6.246159
0                       bonus  20.524645  5.048256
11                     salary  18.003740  2.989183
2             deferred_income  11.320185  0.338413
12          message_poi_ratio   9.816852  3.371674
7         long_term_incentive   9.772104  2.497366
9            restricted_stock   8.694888  2.463442
6               loan_advances   7.125382  6.634816
5                    expenses   5.287549  1.293600
8                       other   4.143788  1.703679
3               director_fees   1.972788  1.508957
13       message_others_ratio   1.473822  0.392669
10  restricted_stock_deferred   0.767702  0.008756
1           deferral_payments   0.236711  0.094344


The two metrics returned similar values of importance for the features. They differed only in the loan_advances and salary records.
I'm happy to know that the feature I created message_poi_ratio got a good score.

# 7. Testing models

In [13]:
models = []
# naive_bayes
models.append(('Naive Bayes', GaussianNB()))
# tree
models.append(('DecisionTree', DecisionTreeClassifier(random_state=42)))
# ensemble
models.append(('RandomForest', RandomForestClassifier(n_estimators=100, random_state=42)))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0)))

models.append(('AdaBoost', AdaBoostClassifier(n_estimators=100, random_state=42)))
models.append(('ExtraTreesClassifier', ExtraTreesClassifier(n_estimators=100, random_state=42)))
# linear_model
models.append(('LogisticRegression', LogisticRegression(random_state=42)))
# neighbors
models.append(('KNeighbors', KNeighborsClassifier(n_neighbors=5)))
models.append(('NearestCentroid', NearestCentroid()))

accuracy_model = []
score = []
for name, clf in models:  
    tm = time()
    #print "Start testing classifier:", name

    score = test_classifier(clf, my_dataset, features_list)
    accuracy_model.append([name,score[2], score[3], score[1]])
    
scores = pd.DataFrame(accuracy_model,
                      columns=('Model', 
                               'Precision', 
                               'Recall',
                               'Accuracy')).sort_values(by='Accuracy',
                                                    ascending = False)

print scores

                        Model  Precision  Recall  Accuracy
7                  KNeighbors   0.705556  0.1270  0.876533
2                RandomForest   0.441931  0.1465  0.861533
5        ExtraTreesClassifier   0.411685  0.1515  0.858000
4                    AdaBoost   0.418319  0.3060  0.850733
8             NearestCentroid   0.376596  0.2655  0.843467
3  GradientBoostingClassifier   0.260177  0.2205  0.812467
1                DecisionTree   0.252969  0.2450  0.802867
6          LogisticRegression   0.164218  0.1900  0.763067
0                 Naive Bayes   0.219135  0.6230  0.653733


As I'm interested in the three metrics recall, precision and accuracy in the test with basic parameters, the Models NearestCentroid and GradientBoostingClassifier had a score close to the expected of 0.3. Even without tuning the AdaBoost reached the metric with a precision of 0.41, recall of 0.30 and a good percentage of acuracia of 85%.

One of the models has reached the goal, but what if we can increase the precision and recall of the other models? Sounds like a good mission, does not it? We, will try.

# 8. Tuning Algorithm Parameters

## 8.1. GradientBoostingClassifier

In [14]:
pipeline = Pipeline([('scale', StandardScaler(with_mean=False)),
                     ('classify', GradientBoostingClassifier(random_state=42))])

param_grid = {
        'classify__criterion': ['mae', 'friedman_mse', 'mse'],
        'classify__n_estimators' : [50, 100, 150],
        'classify__learning_rate':[0.05, 0.1, 1.0],
        'classify__loss' : ['deviance', 'exponential'],    
        'classify__max_leaf_nodes': [100, 150],
        'classify__max_features': [0.50]       
    }
grid = GridSearchCV(
    pipeline, param_grid=param_grid, cv=10, scoring='accuracy')

grid.fit(features,labels)
clf = grid.best_estimator_
test_classifier(clf, my_dataset, features_list, print_result=True)

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=False, with_std=True)), ('classify', GradientBoostingClassifier(criterion='mae', init=None, learning_rate=1.0,
              loss='exponential', max_depth=3, max_features=0.5,
              max_leaf_nodes=100, min_impurity_split=1e-07,
           ...        presort='auto', random_state=42, subsample=1.0, verbose=0,
              warm_start=False))])
	Accuracy: 0.83473	Precision: 0.31391	Recall: 0.20200	F1: 0.24582	F2: 0.21751
	Total predictions: 15000	True positives:  404	False positives:  883	False negatives: 1596	True negatives: 12117



After a scale and a search for best params with gridsearchCV we improve all of three scores, but I can't  able to achieve a recall greater than 0.30.

| Model                        | Precision |    Recall | Accuracy  |
|------------------------------|-----------|-----------|-----------|
|GradientBoosting before       |   0.26017 |     0.2205|    0.81246|
|GradientBoosting after tuning |   0.31391 |     0.2020|    0.83473|

## 8.2. NearestCentroid

In [15]:
pipeline = Pipeline(steps=[('scale', StandardScaler()),
                           ('reduce_dim', PCA(copy=True, random_state=42)),
                           ('classifier', NearestCentroid())
                          ])

param_grid = {'scale' : [StandardScaler(with_mean=False)],
              'reduce_dim__n_components': [6],
              'classifier__metric': ['cityblock', 'euclidean', 'l1', 'l2', 'manhattan'],
              'classifier__shrink_threshold'  : [0.001, 0.01, 0.1, 1.0]
         }

#scv = StratifiedShuffleSplit(labels_train, 1000, random_state = 42)

grid = GridSearchCV(
    pipeline, param_grid=param_grid, cv=10, scoring='accuracy')

grid.fit(features_train, labels_train)

clf = grid.best_estimator_
test_classifier(clf, my_dataset, features_list, print_result=True)

C:\Users\lieby\Anaconda2\lib\site-packages\sklearn\neighbors\nearest_centroid.py:138: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=False, with_std=True)), ('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=6, random_state=42,
  svd_solver='auto', tol=0.0, whiten=False)), ('classifier', NearestCentroid(metric='cityblock', shrink_threshold=0.001))])
	Accuracy: 0.80793	Precision: 0.34899	Recall: 0.50900	F1: 0.41407	F2: 0.46625
	Total predictions: 15000	True positives: 1018	False positives: 1899	False negatives:  982	True negatives: 11101



Now I'm happier with the result, although it has reduced Precision and accuracy a bit, there was a significant increase in recall.

| Model                        | Precision |    Recall | Accuracy  |
|------------------------------|-----------|-----------|-----------|
|NearestCentroid before        |    0.37659|     0.2655|    0.84346|
|NearestCentroid after tuning  |    0.34899|     0.5090|    0.80793|

## 8.3. Last but not least...  AdaBoostClassifier

First, I'll tuning the classify that I'll use after as base estimator on Adaboost.

In [35]:
pipeline = Pipeline(steps=[('minmaxer', StandardScaler(with_mean=False)),
                           ('classifier', DecisionTreeClassifier(random_state=42))
                          ])
params = {
          'classifier__max_features': [2, 6, 10],
          'classifier__min_samples_split' : [0.1, .50, .90],
          'classifier__splitter' : ['best' , 'random'],
          'classifier__min_samples_leaf': [1, 3, 10,15],
          'classifier__class_weight' : ['balanced', None],
          'classifier__criterion': ["gini", "entropy"]
        }

grid = GridSearchCV(pipeline, param_grid = params,scoring = 'recall', cv=10)
grid.fit(features_train, labels_train)

clf_bs = grid.best_estimator_

test_classifier(clf_bs, my_dataset, features_list, print_result=True)

Pipeline(steps=[('minmaxer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=0.5, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='random'))])
	Accuracy: 0.69293	Precision: 0.26790	Recall: 0.75200	F1: 0.39506	F2: 0.55237
	Total predictions: 15000	True positives: 1504	False positives: 4110	False negatives:  496	True negatives: 8890



In [38]:
pipeline = Pipeline(steps=[('minmaxer', MinMaxScaler(feature_range=(0, 1))),
                           #('selection', SelectKBest()),
                           #('reduce',PCA(n_components=6, random_state=42)),
                           ('classifier', AdaBoostClassifier(random_state=42))
                               ])
params = {
          #'selection__k': [14],
          'classifier__base_estimator': [DecisionTreeClassifier(class_weight='balanced', criterion='gini',
                                        max_depth=None, max_features=2, max_leaf_nodes=None,
                                        min_samples_leaf=1,
                                        min_samples_split=0.5, min_weight_fraction_leaf=0.0,
                                        presort=False, random_state=42, splitter='random')], 
          'classifier__n_estimators': [150,200],          
          'classifier__learning_rate' :[0.01, 0.05, 0.1, 1.0],
          'classifier__algorithm' : ['SAMME.R', 'SAMME']
               }

# You can use StratifiedShuffleSplit
#scv = StratifiedShuffleSplit(features_train, 1000, random_state = 42)

# set up gridsearch
grid = GridSearchCV(pipeline, param_grid = params,scoring = 'recall', cv=10)
grid.fit(features_train, labels_train)

clf = grid.best_estimator_

test_classifier(clf, my_dataset, features_list, print_result=True)

Pipeline(steps=[('minmaxer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurit...ndom_state=42, splitter='random'),
          learning_rate=0.1, n_estimators=150, random_state=42))])
	Accuracy: 0.80280	Precision: 0.36377	Recall: 0.63950	F1: 0.46374	F2: 0.55531
	Total predictions: 15000	True positives: 1279	False positives: 2237	False negatives:  721	True negatives: 10763



We got it! I think I found the one ;)

I tried to reduce dimensionality but did not succeed. So I chose not to use PCA or Kbest.
This model achieves the goal of a score greater than 0.30 for recall and precision.


| Model                           | Precision |    Recall | Accuracy  |
|---------------------------------|-----------|-----------|-----------|
|AdaBoostClassifier before        |    0.41831|     0.3060|    0.85073|
|AdaBoostClassifier after tuning  |    0.36377|     0.6395|    0.80280|

## Now I'm a bit curious...
about how much other models can be improved. Let's continue the tests.

## 8.4. RandomForestClassifier

In [28]:
pipeline =   Pipeline([('scale', MinMaxScaler(feature_range=(0, 1))),
                       ('classifier', RandomForestClassifier())
                      ])

param_grid = {'scale': [None, MaxAbsScaler(), MinMaxScaler(feature_range=(0, 1))],
              'classifier__max_depth': [5, 3, 1],
              'classifier__max_features': [2],
              'classifier__min_samples_leaf': [1, 3, 10,15],
              'classifier__bootstrap': [True, False],
              'classifier__criterion': ['gini', 'entropy']
             }

grid = GridSearchCV(pipeline, param_grid=param_grid, cv=10, scoring='recall')

grid.fit(features_train, labels_train)

clf =  grid.best_estimator_
test_classifier(clf, my_dataset, features_list, print_result=True)

Pipeline(steps=[('scale', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=2, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])
	Accuracy: 0.86080	Precision: 0.43103	Recall: 0.13750	F1: 0.20849	F2: 0.15918
	Total predictions: 15000	True positives:  275	False positives:  363	False negatives: 1725	True negatives: 12637



No significant improvement:

| Model                           | Precision |    Recall | Accuracy  |
|---------------------------------|-----------|-----------|-----------|
|RandomForest before              |    0.44193|     0.1465|    0.86153|
|RandomForest after tuning        |    0.43103|     0.1375|    0.86080|

## 8.5. KNeighborsClassifier

In [16]:
pipeline = Pipeline(steps=[('scale', StandardScaler(with_mean=False)),
                           #('reduce_dim', PCA(copy=True, random_state=42)),
                           #('selection', SelectKBest()),
                           ('classifier', KNeighborsClassifier())
                          ])

param_grid = {'scale' : [None, StandardScaler(with_mean=False)],
              #'selection__k': [6, 10, 14],
              #'reduce_dim__n_components': [10],
              'classifier__metric': ['minkowski','euclidean'],
              'classifier__n_neighbors' : range(2,14,2),
              'classifier__weights' : ['uniform','distance']
             }

grid = GridSearchCV(
    pipeline, param_grid=param_grid, cv=10, scoring='recall')

grid.fit(features_train, labels_train)

clf = grid.best_estimator_
test_classifier(clf, my_dataset, features_list, print_result=True)

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=False, with_std=True)), ('classifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='distance'))])
	Accuracy: 0.83980	Precision: 0.39456	Recall: 0.37700	F1: 0.38558	F2: 0.38039
	Total predictions: 15000	True positives:  754	False positives: 1157	False negatives: 1246	True negatives: 11843



This classifier had a decrease in the recall score. Now it has a more balanced values for precision and recall.

| Model                           | Precision |    Recall | Accuracy  |
|---------------------------------|-----------|-----------|-----------|
|KNeighbors before                |    0.70555|     0.1270|    0.87653|
|KNeighbors after tuning          |    0.39456|     0.3770|    0.83980|

## 8.6. LogisticRegression

In [29]:
pipeline = Pipeline(steps=[('minmaxer', MinMaxScaler(feature_range=(0, 1))),
                           #('minmaxer', StandardScaler(with_mean=False)),
                           #('reduce_dim', PCA(copy=True, random_state=42)),
                           #('selection', SelectKBest()),
                           ('classifier', LogisticRegression(random_state=42))
                          ])

param_grid = {#'minmaxer' : [StandardScaler(with_mean=False), None],
              #'selection__k': [10, 'all'],
              'classifier__C': [0.05, 0.5, 1, 10, 100, 500, 1000],
              'classifier__solver': ['liblinear'],
              'classifier__penalty': ['l2'], 
              'classifier__class_weight': ['balanced']
             }

grid = GridSearchCV(pipeline, param_grid=param_grid, cv=10, scoring='accuracy')

grid.fit(features_train, labels_train)
clf = grid.best_estimator_
test_classifier(clf, my_dataset, features_list, print_result=True)

Pipeline(steps=[('minmaxer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', LogisticRegression(C=500, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])
	Accuracy: 0.76647	Precision: 0.30177	Recall: 0.57200	F1: 0.39510	F2: 0.48512
	Total predictions: 15000	True positives: 1144	False positives: 2647	False negatives:  856	True negatives: 10353



I really liked the improvement achieved in this classifier after tuning.

| Model                           | Precision |    Recall | Accuracy  |
|---------------------------------|-----------|-----------|-----------|
|LogisticRegression  before       |    0.16421|     0.1900|    0.76306|
|LogisticRegression  after tuning |    0.30177|     0.5720|    0.76647|

## 8.7. ExtraTreesClassifier

In [32]:
pipeline = Pipeline(steps=[('minmaxer', MinMaxScaler(feature_range=(0, 1))),
                           ('classifier', ExtraTreesClassifier(random_state=42))
                          ])
params = {
          'classifier__max_features': [2, 6, 10],
          'classifier__min_samples_split' : [0.1, .50, .90],
          'classifier__min_samples_leaf': [1, 3, 10,15],
          'classifier__class_weight' : ['balanced', None],
          'classifier__criterion': ["gini", "entropy"]
        }

grid = GridSearchCV(pipeline, param_grid = params,scoring = 'recall', cv=10)
grid.fit(features_train, labels_train)

clf_bs = grid.best_estimator_

test_classifier(clf_bs, my_dataset, features_list, print_result=True)

Pipeline(steps=[('minmaxer', MinMaxScaler(copy=True, feature_range=(0, 1))), ('classifier', ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
           criterion='gini', max_depth=None, max_features=2,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=10, min_samples_split=0.1,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])
	Accuracy: 0.63640	Precision: 0.22082	Recall: 0.68300	F1: 0.33374	F2: 0.48146
	Total predictions: 15000	True positives: 1366	False positives: 4820	False negatives:  634	True negatives: 8180



# Conclusion